In [1]:
import sys
import pprint
import spotipy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from sklearn.cluster import KMeans

# Connect to spotify an give proper permissions for playlist customization
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id="7444209f2bc34f7ba86b8ce6d7fe5ab7",
        client_secret="c2b5526ba53c46cdb770351224ce9612",
        redirect_uri="http://localhost:8080",
        scope="user-top-read,user-library-read,user-read-recently-played,playlist-modify-private,playlist-modify-public",
    )
)

# Organize NACC chart into needed columns and group by category
keep = ["Artist", "Recording", "Label"]
df = pd.read_csv("test.csv", usecols=keep)
albums = df.groupby("Recording")

# Establish main user (only me for now)
user_id = sp.me()["id"]

# Create Playlist
playlist = sp.user_playlist_create(user_id, "NACC Top 30", "Test")

# Go through each artist on NACC chart, search via Spotipy and add each song off the top result
for index, row in df.iterrows():

    # Get album name
    search = row["Recording"]
    artist = row["Artist"]

    # search spotify for album
    query = f"album:{search} artist:{artist}"
    result = sp.search(q=query, limit=1, type="album", market="US")
    search_result = result["albums"]["items"]
    # Check if spotify was able to find the album
    if not search_result:
        print("Couldn't find",search,"by",artist)
        continue

        # Spotify couln't find the album
    else:
        print(
            search_result[0]["name"],
            "by",
            search_result[0]["artists"][0]["name"],
            "added, Spotify ID:",
            search_result[0]["id"],
        )

        # Organize search results for playlist
        tracks = sp.album_tracks(search_result[0]["id"])
        songs = tracks["items"]

        # Add songs to playlist
        for track in songs:
            sp.playlist_add_items(playlist_id=playlist["id"], items=[track["id"]])

playlist_tracks = sp.playlist_tracks(playlist["id"])
tracks = playlist_tracks["items"]
track_ids = [track["track"]["id"] for track in tracks]
audio_features = sp.audio_features(track_ids)

# Generate data frame of audio features

print(audio_features)

audio_features_df = pd.DataFrame(audio_features, columns=audio_features[0].keys())
audio_features_df = audio_features_df[
            [
                "danceability",
                "energy",
                "speechiness",
                "acousticness",
                "instrumentalness",
                "liveness",
                "valence",
                "tempo",
            ]
        ]

# Create correlation matrix of audio features
matrix = audio_features_df.corr()

# Display matrix
sns.heatmap(matrix, annot=True, cmap="coolwarm")

Charm by Clairo added, Spotify ID: 1KNUCVXgIxKUGiuEB8eG0i
Dr. Dog by Dr. Dog added, Spotify ID: 4gjV2tn14HAQusWUf4GKtl


KeyboardInterrupt: 